In [1]:
#Import pyomo
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
# Data
#District
D = {1,2,3}
S = {'Female', 'Male'}
# Female and male students for each district
f = {1: 210, 2: 210, 3:180}
m = {1: 120, 2: 30, 3:150}
#Distances between districs
d = {(1,1): 0, (1,2): 3, (1,3):5,
    (2,1):3, (2,2): 0, (2,3):4,
     (3,1):5 , (3,2):4, (3,3):0
    }

In [3]:
# model and sets
model = pe.ConcreteModel()
model.D = pe.Set(initialize = D, ordered = False)
model.S = pe.Set(initialize = S, ordered = False)

In [4]:
## Decision variables
model.x = pe.Var(model.D, model.D, model.S, domain = pe.NonNegativeIntegers)

In [5]:
## Objective function
objExpr = sum(model.x[i,j,s] * d[i,j] for i in model.D for j in model.D for s in model.S )
model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [6]:
## Constraints

#First set of constraints
model.conList1 = pe.ConstraintList()
for i in model.D: 
    expr = sum(model.x[i,j,'Female']  for j in model.D) == f[i]
    model.conList1.add(expr)

#Second set of constraints
model.conList2 = pe.ConstraintList()
for i in model.D: 
    expr = sum(model.x[i,j,'Male']  for j in model.D) == m[i]
    model.conList2.add(expr)

# Third set of constraints
model.conList3 = pe.ConstraintList()
for j in model.D:
    expr = sum(model.x[i,j,k] for i in model.D for k in model.S) == sum(f[i] + m[i] for i in model.D) / len(model.D)
    model.conList3.add(expr)

#Fourth set of constraints
model.conList4 = pe.ConstraintList()
for j in model.D: 
    expr = sum (model.x[i,j,'Male'] for i in model.D) == sum(m[i] for i in model.D) / len(model.D)
    model.conList4.add(expr)

In [7]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-12-10
Read LP format model from file C:\Users\rpo330\AppData\Local\Temp\tmp2nla26uf.pyomo.lp
Reading time = 0.00 seconds
x19: 13 rows, 19 columns, 46 nonzeros
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 13 rows, 19 columns and 46 nonzeros
Model fingerprint: 0x69a380fc
Variable types: 1 continuous, 18 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 1870.0000000
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Found heuristic solution: objective 710.0000000
Variable types: 0 continuous, 9 integer (0 binary)

Root relaxation: objective 3.500000e+02, 2 iterations, 0.00 seconds (0.0

In [8]:
for i in model.D:
     for j in model.D:
            for s in model.S:
                if pe.value(model.x[i,j,s]) > 0:
                    print(str(model.x[i,j,s])+" = "+ str(pe.value(model.x[i,j,s])))

x[1,1,Female] = 200.0
x[1,1,Male] = 100.0
x[1,2,Male] = 20.0
x[1,3,Female] = 10.0
x[2,2,Female] = 200.0
x[2,2,Male] = 30.0
x[2,3,Female] = 10.0
x[3,2,Male] = 50.0
x[3,3,Female] = 180.0
x[3,3,Male] = 100.0
